<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     Software Development with Python
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:55%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:62%; left:10%;">
    Data Scientist
</h3>
</div>

# Frames and traceback

In Python, nearly everything is available for introspection.  One of the design concepts in Python that you should understand is that execution is arranged as a *stack* of nested *frames* (this design is shared by many other programming languages as well).  A particular function or method has its own local variables and its own execution steps.  These steps are expressed by programmers as lines of source code, but internally Python reduces these lines to *word-codes* (often called *byte-codes* for historical reasons, but not accurate since Python 3.6).

When one function calls another function, that generates a new frame, and the flow of control passes to that child frame.  After a while (generally) that child frame terminates and returns some value.  Flow then continues within the parent frame.  The parent likely terminates and returns a value and flow control to the grandparent, and so on.  Eventually, the top level of the program overall ends, and the Python interpreters stops running.

When exceptions occur, tracebacks, by default, are printed to standard error.  Tracebacks display a stack of frames, going from where the exception actually occurs up to some frame where the exception is caught.  If no frame catches the exception, the Python interpreter itself displays this traceback before terminating.

## Examining frames

Let us illustrate this with a very simple program, first as a script in a file, then as a Jupyter notebook cell.  By the time a notebook cell runs, we are already rather deep into a call stack.  I use the function `sys._getframe()` here; this is technically an implementation detail of CPython, and may not exist in other implementations of Python.

In [1]:
!cat -n tb.py
print('-----')
!python tb.py

     1	import sys
     2	for i in range(sys.getrecursionlimit()):
     3	    try:
     4	        print(f"{i+1}: {sys._getframe(i)}")
     5	    except:
     6	        break
-----
1: <frame at 0x7f94d3fae440, file 'tb.py', line 4, code <module>>


Running identical code inside a notebook cell:

In [2]:
import sys
for i in range(sys.getrecursionlimit()):
    try:
        print(f"{i+1}: {sys._getframe(i)}")
    except:
        break

1: <frame at 0x7fb6705a7230, file '<ipython-input-2-10b239d7e237>', line 4, code <module>>
2: <frame at 0x55774e187f60, file '/home/dmertz/miniconda3/envs/INE/lib/python3.8/site-packages/IPython/core/interactiveshell.py', line 3418, code run_code>
3: <frame at 0x55774e16e870, file '/home/dmertz/miniconda3/envs/INE/lib/python3.8/site-packages/IPython/core/interactiveshell.py', line 3338, code run_ast_nodes>
4: <frame at 0x55774e1707f0, file '/home/dmertz/miniconda3/envs/INE/lib/python3.8/site-packages/IPython/core/interactiveshell.py', line 3146, code run_cell_async>
5: <frame at 0x7fb670545220, file '/home/dmertz/miniconda3/envs/INE/lib/python3.8/site-packages/IPython/core/async_helpers.py', line 68, code _pseudo_sync_runner>
6: <frame at 0x55774e1878c0, file '/home/dmertz/miniconda3/envs/INE/lib/python3.8/site-packages/IPython/core/interactiveshell.py', line 2923, code _run_cell>
7: <frame at 0x7fb67054d040, file '/home/dmertz/miniconda3/envs/INE/lib/python3.8/site-packages/IPython/co

## Introspecting frames and stacks

We can look at the Python stack within a Python program itself.  This is part of what debuggers do, for example.  We are able to code our own logic to examine them though.  Often logging this kind of information can be useful.  In this lesson, I merely print it off.  At times, you may want your functions to make actual decisions based on the the stack of frames leading down to them.  However, so-called "frame hacking" should be used with caution, and is frowned upon when more explicit data passing is available.

In [3]:
import traceback
from collections import namedtuple
fn = namedtuple('FUN', ['name', 'spam'])

Here is a general purpose function that, when reached, will print off information about its ancestor frames.  Just for illustration, we look at how such an ancestor frame has defined its own `spam` variable.  This example is trivial, but having some complex data object with the same name, contain something different at different levels is a common source of bugs and confusion.

In [4]:
def how_did_I_get_here():
    "Function that reports on that call stack leading to it"
    spam = "now"
    
    # Hack to make this function sometimes raise an exception
    if globals().get('RAISE'):
        raise NotImplementedError("How did I get here?")
        
    # Self report on frame stack
    f = sys._getframe()
    path = [fn(f.f_code.co_name, f.f_locals.get('spam'))]
    # Minor hack because notebook cells are many levels into stack already
    while not f.f_code.co_name.startswith("start"):
        f = f.f_back
        path.append(fn(f.f_code.co_name, f.f_locals.get('spam')))
    for n, frame in enumerate(reversed(path)):
        print("  "*n, "⤷", frame, sep='')

Let's define a function that will launch a stack of nested calls, but in varying ways.

In [5]:
def start(lang='EN'):
    if lang == 'EN':
        one()
    elif lang == 'ES':
        uno()
    else:
        pass

Define a path of nested calls.

In [6]:
def one(spam=1): 
    two()
    
def two(spam=2): 
    three()
    
def three(spam=3): 
    how_did_I_get_here()

Definir una ruta de funciones anidadas

In [7]:
def uno(spam=1):
    dos()
    
def dos(spam=2):
    tres()
    
def tres(spam=3):
    cuatro()
    
def cuatro(spam=4):
    how_did_I_get_here()

In [8]:
start('EN')

⤷FUN(name='start', spam=None)
  ⤷FUN(name='one', spam=1)
    ⤷FUN(name='two', spam=2)
      ⤷FUN(name='three', spam=3)
        ⤷FUN(name='how_did_I_get_here', spam='now')


In [9]:
start('ES')

⤷FUN(name='start', spam=None)
  ⤷FUN(name='uno', spam=1)
    ⤷FUN(name='dos', spam=2)
      ⤷FUN(name='tres', spam=3)
        ⤷FUN(name='cuatro', spam=4)
          ⤷FUN(name='how_did_I_get_here', spam='now')


In [10]:
[attr for attr in dir(sys._getframe()) if attr.startswith('f_')]

['f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

## Working with tracebacks

Just as we may want to look "up" the stack within a nested call, we might want to look "down" the stack when an exception occurs in such a nested call.  The `traceback` module allows us to do that.

Let's first look at a standard traceback to remind ourselves.

In [11]:
RAISE = True
start()

NotImplementedError: How did I get here?

There are a few different ways to interrogate the traceback.  One is using the error object itself, once we catch it.

In [12]:
def start2():
    try:
        one()
    except Exception as err:
        # Can pull the traceback out of the exception object
        traceback.print_tb(err.__traceback__)
        
start2()

  File "<ipython-input-12-f3221e149481>", line 3, in start2
    one()
  File "<ipython-input-6-34b62cebd10f>", line 2, in one
    two()
  File "<ipython-input-6-34b62cebd10f>", line 5, in two
    three()
  File "<ipython-input-6-34b62cebd10f>", line 8, in three
    how_did_I_get_here()
  File "<ipython-input-4-becdce95cbde>", line 7, in how_did_I_get_here
    raise NotImplementedError("How did I get here?")


The same information is available inherently for `sys.exc_info()` even if we do not capture the error object.

In [13]:
def start3():
    try:
        one()
    except:
        # Can query sys.exc_info()
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print(f"Exception class: {exc_type.__name__}")
        print(f"Exception message: {exc_value}")
        for frame, lineno in traceback.walk_tb(exc_traceback):
            print(f"Line: {lineno} | Function: {frame.f_code.co_name}")

start3()

Exception class: NotImplementedError
Exception message: How did I get here?
Line: 3 | Function: start3
Line: 2 | Function: one
Line: 5 | Function: two
Line: 8 | Function: three
Line: 7 | Function: how_did_I_get_here


With very little extra work, we can replicate the call graph display we had above, but as a caught exception rather than a nested stack examination.  That is, earlier we looked **up** the call stack.  Here we look **down** the traceback.

In [14]:
def start4():
    try:
        one()
    except Exception as err:
        for n, (f, _) in enumerate(traceback.walk_tb(err.__traceback__)):
            frame = fn(f.f_code.co_name, f.f_locals.get('spam'))
            print("  "*n, "⤷", frame, sep='')
            
start4()

⤷FUN(name='start4', spam=None)
  ⤷FUN(name='one', spam=1)
    ⤷FUN(name='two', spam=2)
      ⤷FUN(name='three', spam=3)
        ⤷FUN(name='how_did_I_get_here', spam='now')


We have managed to perform custom inspection of local data within each descendant frame that eventually led to an exception.  Looking at this changing data (here, the simplified `spam` variable) may help us better diagnose program state when we encountered a problem.